# Multinomial Naive Bayes Text Classifier

In [1]:
# imports

import numpy as np
import pandas as pd
import nltk
import sklearn

The MNB Classifier (courtesy of ML hw1)

In [2]:
class MNB_TextClassifier:
    
    def __init__(self):
        self.prior = {}
        self.condprob = {}
    
    # p(class=y | {term0=x[0], term1=x[1], ...} )
    # please note that you dont add new data to vocabulary when predicting
    def proba_y_given_x(self, y, x):
        length = len(self.condprob)
        prob_y = self.prior[y]
        prob_x = 1
        for i in x:
            if(i in self.condprob.keys()):
                prob_x *= (self.condprob[i][y])
            else:
                prob_x *= ((1)/(self.dict_y_term[y] + length))
        return prob_y * prob_x
 
        #raise NotImplementedError()
    
    # p(c)
    def proba_y(self, y):
        return self.prior[y]

    # update self.prior[class] as p(class=class) 
    # update self.condprob[term][class] as p(term=term | class=class)
    def fit(self, X, y):
        dict_y = {i:list(y).count(i) for i in y}
        total_y = 0

        for i in dict_y:
            total_y += dict_y[i]
        for i in dict_y:
            self.prior[i] = dict_y[i]/total_y
        
        X_Plain = list()
        
        for i in X:
            for j in i:
                X_Plain.append(j)

        dict_x = {i:X_Plain.count(i) for i in X_Plain}
        dict_x_dict_y = {i:dict() for i in dict_x}
        dict_x_cond_y = {i:dict() for i in dict_x}

        for i in range(len(X)):
            for j in X[i]:
                if(i in dict_x_cond_y[j]):
                    dict_x_cond_y[j][i] += 1
                else:
                    dict_x_cond_y[j][i] = 1
        self.dict_y_term = {i:0 for i in y}
        for i in range(len(y)):
            self.dict_y_term[y[i]] += (len(X[i]))

        dict_y_last = {i:{j:0 for j in self.dict_y_term} for i in dict_x_cond_y}

        for i in dict_x_cond_y:
            for j in range(len(y)):
                if(j in dict_x_cond_y[i]):
                    dict_y_last[i][y[j]] += dict_x_cond_y[i][j] 
        self.condprob = {i:{j:0 for j in self.dict_y_term} for i in dict_x_cond_y}
        for i in dict_y_last:
            for j in dict_y_last[i]:

                self.condprob[i][j] = (dict_y_last[i][j] + 1)/(self.dict_y_term[j] + len(dict_x))
        #print(dict_x)
        #print(self.dict_y_term)
        #print(dict_y_last)
        #raise NotImplementedError()
        
    def predict_single(self, x):
        classes = [i for i in self.prior.keys()]
        likelihood = [self.proba_y_given_x(i,x) for i in classes]
        return classes[likelihood.index(max(likelihood))]
        #raise NotImplementedError()
        
    def predict(self, X):
        return [self.predict_single(x) for x in X]

Using single annotator csv as <b>TRAIN DATA</b>

In [4]:
single_annotator = pd.read_csv("training_clean/clean_single_annotator.csv",delimiter=",")
single_annotator.head()

,Unnamed: 0,kalimat_id,kata,sense,kalimat
0,0,336691,cerah,4801,cuaca cerah lazim
1,1,336270,cerah,4801,gambar dihasilkan layarnya cerah memiliki spea...
2,2,336555,cerah,4803,cerah pemuda berumur 20 prancis abad 17
3,3,336618,cerah,4801,cor caroli alpha canum venaticorum nama lengka...
4,4,336613,cerah,4801,sanders menyukai cat air lilo maksud menampilk...


In [25]:
# This is creating the MNB Text Classifier and fitting X to y

df = single_annotator[single_annotator['kata']=='cerah']

arr = [nltk.word_tokenize(df['kalimat'].iloc[i]) for i in range(len(df))]

X = np.array(arr)

y = np.array(df['sense'].values)

clf = MNB_TextClassifier()
clf.fit(X, y)
clf.proba_y_given_x('4801',['cuaca']) #test proba
clf.predict_single('cuaca hari ini cerah boyyyyyyy')

'4802'

Using double annotator agree csv as <b>TEST DATA</b>

In [64]:
double_annotator_agree = pd.read_csv("training_clean/clean_double_annotator_agree.csv",delimiter=",")

df_test = double_annotator_agree[double_annotator_agree['kata']=='cerah']

test_arr = [nltk.word_tokenize(df_test['kalimat'].iloc[i]) for i in range(len(df_test))]
test_X = np.array(test_arr)

test_y = np.array(df_test['sense'].values)

y_pred = np.array(clf.predict(test_X))

acc = sklearn.metrics.accuracy_score(test_y, 
                                     y_pred)

acc

0.8421052631578947

Now let's try our model with the actual TEST DATA

In [26]:
test_data = pd.read_csv("testing_set/clean_testing_data.csv",delimiter=",")
test_data.head()

,Unnamed: 0,id,word,kalimat
0,0,13,asing,pecinta film indonesia tv asing mendengar namanya
1,1,19,asing,telinga asing aneh mendengar menu masakan soto...
2,2,41,asing,warga negara asing warga negara persemakmuran ...
3,3,44,asing,belas memerintah sultan mahmud menjalin kerja ...
4,4,121,asing,diikuti donatdonat waralaba asing master ring ...


In [27]:
words = test_data.word.unique()
words

array(['asing', 'atas', 'badan', 'baru', 'berat', 'besar', 'bidang',
       'bintang', 'bisa', 'buah', 'bulan', 'bunga', 'cabang', 'cerah',
       'coklat', 'dalam', 'dasar', 'dunia', 'halaman', 'harapan', 'jalan',
       'jam', 'jaringan', 'kabur', 'kaki', 'kali', 'kepala', 'ketat',
       'kulit', 'kunci', 'layar', 'lebat', 'lingkungan', 'mata',
       'membawa', 'memecahkan', 'menangkap', 'mendorong', 'menerima',
       'mengandung', 'mengejar', 'mengeluarkan', 'mengikat', 'mengisi',
       'menjaga', 'menurunkan', 'menyusun', 'nilai', 'panas', 'pembagian',
       'rapat', 'sarung', 'tengah', 'tinggi'], dtype=object)

Still using single annotator as TRAIN DATA
Here we are reimporting

In [28]:
single_annotator = pd.read_csv("training_clean/clean_single_annotator.csv",delimiter=",")
single_annotator.head()

,Unnamed: 0,kalimat_id,kata,sense,kalimat
0,0,336691,cerah,4801,cuaca cerah lazim
1,1,336270,cerah,4801,gambar dihasilkan layarnya cerah memiliki spea...
2,2,336555,cerah,4803,cerah pemuda berumur 20 prancis abad 17
3,3,336618,cerah,4801,cor caroli alpha canum venaticorum nama lengka...
4,4,336613,cerah,4801,sanders menyukai cat air lilo maksud menampilk...


For each word, make an MNB classifier, and write to a new dataframe

In [30]:
answer = pd.DataFrame(columns=['kalimat_id', 'kata', 'sense'])

all_rows = []

#do the same as MNB above, but making one MNB for the specific word, and iterating the test data
#for trying a different model, this part needs to be edited. Replace the MNB classifier with something else
for word in words:
    df = single_annotator[single_annotator['kata']==word]
    
    arr = [nltk.word_tokenize(df['kalimat'].iloc[i]) for i in range(len(df))]
    
    X = np.array(arr)

    y = np.array(df['sense'].values)

    clf = MNB_TextClassifier()
    clf.fit(X, y)
    
    test_current = test_data[test_data['word']==word]
    
    for index, row in test_current.iterrows():
        sense = clf.predict_single(row['kalimat'])
        row_dict = {'kalimat_id':row['id'],
                    'kata':word,
                    'sense':sense}
        all_rows.append(row_dict)

In [31]:
all_rows

[{'kalimat_id': 13, 'kata': 'asing', 'sense': '5303'},
 {'kalimat_id': 19, 'kata': 'asing', 'sense': '5303'},
 {'kalimat_id': 41, 'kata': 'asing', 'sense': '5302'},
 {'kalimat_id': 44, 'kata': 'asing', 'sense': '5303'},
 {'kalimat_id': 121, 'kata': 'asing', 'sense': '5303'},
 {'kalimat_id': 138, 'kata': 'asing', 'sense': '5303'},
 {'kalimat_id': 139, 'kata': 'asing', 'sense': '5303'},
 {'kalimat_id': 140, 'kata': 'asing', 'sense': '5303'},
 {'kalimat_id': 188, 'kata': 'asing', 'sense': '5302'},
 {'kalimat_id': 190, 'kata': 'asing', 'sense': '5302'},
 {'kalimat_id': 205, 'kata': 'asing', 'sense': '5303'},
 {'kalimat_id': 242, 'kata': 'asing', 'sense': '5302'},
 {'kalimat_id': 286, 'kata': 'asing', 'sense': '5302'},
 {'kalimat_id': 311, 'kata': 'asing', 'sense': '5302'},
 {'kalimat_id': 313, 'kata': 'asing', 'sense': '5301'},
 {'kalimat_id': 353, 'kata': 'asing', 'sense': '5303'},
 {'kalimat_id': 412, 'kata': 'asing', 'sense': '5303'},
 {'kalimat_id': 419, 'kata': 'asing', 'sense': '5303

In [43]:
hasil_df = pd.DataFrame(all_rows,columns=['kalimat_id', 'kata', 'sense'])
hasil_df.head()

,kalimat_id,kata,sense
0,13,asing,5303
1,19,asing,5303
2,41,asing,5302
3,44,asing,5303
4,121,asing,5303


In [45]:
hasil_df.to_csv("hasil_percobaan/hasil_MNB_text_classifier.csv",header=None,index=False)

## SANDBOX

In [11]:
#single experiment, 4 samples

a = nltk.word_tokenize(single_annotator['kalimat'][0])
b = nltk.word_tokenize(single_annotator['kalimat'][1])
c = nltk.word_tokenize(single_annotator['kalimat'][2])
d = nltk.word_tokenize(single_annotator['kalimat'][3])

X = np.array([
    a,
    b,
    c,
    d,
])

y = np.array([1,1,3,1])

clf = MNB_TextClassifier()
clf.fit(X, y)

X

array([list(['cuaca', 'cerah', 'lazim']),
       list(['gambar', 'dihasilkan', 'layarnya', 'cerah', 'memiliki', 'speaker', 'menghasilkan', 'suara', 'keras', 'jernih']),
       list(['cerah', 'pemuda', 'berumur', '20', 'prancis', 'abad', '17']),
       list(['cor', 'caroli', 'alpha', 'canum', 'venaticorum', 'nama', 'lengkapnya', 'cor', 'caroli', 'regis', 'martyris', 'bintang', 'cerah', 'rasi', 'canes', 'venatici'])],
      dtype=object)

In [13]:
clf.condprob['cerah']

{1: 0.06666666666666667, 3: 0.05263157894736842}

In [9]:
test_data = pd.read_csv("testing_set/testing_data.csv",delimiter=",")
test_data.head()

,id,word,kalimat
0,13,asing,"Para pecinta film indonesia atau tv, pasti tak..."
1,19,asing,Pasti telinga kita merasa asing dan aneh mende...
2,41,asing,Warga negara asing atau warga negara Persemakm...
3,44,asing,"Selama lima belas tahun memerintah, Sultan Mah..."
4,121,asing,Yang kemudian diikuti dengan donat-donat waral...


In [16]:
for index, row in test_data.iterrows():
    print(row['word'])

asing
asing
asing
asing
asing
asing
asing
asing
asing
asing
asing
asing
asing
asing
asing
asing
asing
asing
asing
asing
asing
asing
asing
asing
asing
asing
asing
asing
asing
asing
asing
asing
asing
asing
asing
asing
asing
asing
asing
asing
asing
asing
asing
asing
asing
asing
asing
asing
asing
asing
asing
asing
asing
asing
asing
asing
asing
asing
asing
asing
asing
asing
asing
asing
asing
asing
asing
asing
asing
asing
asing
asing
asing
asing
asing
asing
asing
asing
asing
asing
asing
asing
asing
asing
asing
asing
asing
asing
asing
asing
asing
asing
asing
asing
asing
asing
asing
asing
asing
asing
asing
asing
asing
asing
asing
asing
asing
asing
asing
asing
asing
asing
asing
asing
asing
asing
asing
asing
asing
asing
asing
asing
asing
asing
asing
asing
asing
asing
asing
asing
asing
asing
asing
asing
asing
asing
asing
asing
asing
asing
asing
atas
atas
atas
atas
atas
atas
atas
atas
atas
atas
atas
atas
atas
atas
atas
atas
atas
atas
atas
atas
atas
atas
atas
atas
atas
atas
atas
atas
atas
atas
atas

bisa
bisa
bisa
bisa
bisa
bisa
bisa
bisa
bisa
bisa
bisa
bisa
bisa
bisa
bisa
bisa
bisa
bisa
bisa
bisa
bisa
bisa
bisa
bisa
bisa
bisa
bisa
bisa
bisa
bisa
bisa
bisa
bisa
bisa
bisa
bisa
bisa
bisa
bisa
bisa
bisa
bisa
bisa
bisa
bisa
bisa
bisa
bisa
bisa
bisa
bisa
bisa
bisa
bisa
bisa
bisa
bisa
bisa
bisa
bisa
bisa
bisa
bisa
bisa
bisa
bisa
bisa
bisa
bisa
bisa
bisa
bisa
bisa
bisa
bisa
bisa
bisa
bisa
bisa
bisa
bisa
bisa
bisa
bisa
bisa
bisa
bisa
bisa
bisa
bisa
bisa
bisa
bisa
bisa
bisa
bisa
bisa
bisa
bisa
bisa
bisa
bisa
bisa
bisa
bisa
bisa
bisa
bisa
bisa
bisa
bisa
bisa
bisa
bisa
bisa
bisa
bisa
bisa
bisa
bisa
bisa
bisa
bisa
bisa
bisa
bisa
bisa
bisa
bisa
bisa
bisa
bisa
bisa
bisa
bisa
bisa
bisa
bisa
bisa
bisa
bisa
bisa
bisa
bisa
bisa
bisa
bisa
bisa
bisa
bisa
bisa
bisa
bisa
bisa
bisa
bisa
bisa
bisa
bisa
bisa
buah
buah
buah
buah
buah
buah
buah
buah
buah
buah
buah
buah
buah
buah
buah
buah
buah
buah
buah
buah
buah
buah
buah
buah
buah
buah
buah
buah
buah
buah
buah
buah
buah
buah
buah
buah
buah
buah
buah
buah


halaman
halaman
halaman
halaman
halaman
halaman
halaman
halaman
halaman
halaman
halaman
halaman
halaman
halaman
halaman
halaman
halaman
halaman
halaman
halaman
halaman
halaman
halaman
harapan
harapan
harapan
harapan
harapan
harapan
harapan
harapan
harapan
harapan
harapan
harapan
harapan
harapan
harapan
harapan
harapan
harapan
harapan
harapan
harapan
harapan
harapan
harapan
harapan
harapan
harapan
harapan
harapan
harapan
harapan
harapan
harapan
harapan
harapan
harapan
harapan
harapan
harapan
harapan
harapan
harapan
harapan
harapan
harapan
harapan
harapan
harapan
harapan
harapan
harapan
harapan
harapan
harapan
harapan
harapan
harapan
harapan
harapan
harapan
harapan
harapan
harapan
harapan
harapan
harapan
harapan
harapan
harapan
harapan
harapan
harapan
harapan
jalan
jalan
jalan
jalan
jalan
jalan
jalan
jalan
jalan
jalan
jalan
jalan
jalan
jalan
jalan
jalan
jalan
jalan
jalan
jalan
jalan
jalan
jalan
jalan
jalan
jalan
jalan
jalan
jalan
jalan
jalan
jalan
jalan
jalan
jalan
jalan
jalan
jalan
jala

kunci
kunci
kunci
kunci
kunci
kunci
kunci
kunci
kunci
kunci
kunci
kunci
kunci
kunci
kunci
kunci
kunci
kunci
kunci
kunci
kunci
kunci
kunci
kunci
kunci
kunci
kunci
kunci
kunci
kunci
kunci
kunci
kunci
kunci
kunci
kunci
kunci
kunci
kunci
kunci
kunci
kunci
kunci
kunci
layar
layar
layar
layar
layar
layar
layar
layar
layar
layar
layar
layar
layar
layar
layar
layar
layar
layar
layar
layar
layar
layar
layar
layar
layar
layar
layar
layar
layar
layar
layar
layar
layar
layar
layar
layar
layar
layar
layar
layar
layar
layar
layar
layar
layar
layar
layar
layar
layar
layar
layar
layar
lebat
lebat
lebat
lebat
lebat
lebat
lebat
lebat
lebat
lebat
lebat
lebat
lebat
lebat
lebat
lebat
lebat
lebat
lebat
lebat
lebat
lebat
lebat
lebat
lebat
lebat
lebat
lebat
lebat
lebat
lebat
lebat
lebat
lebat
lebat
lebat
lebat
lebat
lebat
lebat
lebat
lebat
lingkungan
lingkungan
lingkungan
lingkungan
lingkungan
lingkungan
lingkungan
lingkungan
lingkungan
lingkungan
lingkungan
lingkungan
lingkungan
lingkungan
lingkungan
lingkun

mengikat
mengikat
mengikat
mengikat
mengikat
mengikat
mengikat
mengikat
mengikat
mengikat
mengikat
mengikat
mengikat
mengikat
mengikat
mengikat
mengikat
mengikat
mengikat
mengikat
mengikat
mengikat
mengikat
mengikat
mengikat
mengikat
mengikat
mengikat
mengikat
mengikat
mengikat
mengikat
mengikat
mengikat
mengikat
mengikat
mengikat
mengikat
mengikat
mengikat
mengikat
mengikat
mengikat
mengikat
mengikat
mengikat
mengikat
mengikat
mengikat
mengikat
mengikat
mengikat
mengikat
mengikat
mengikat
mengikat
mengikat
mengikat
mengikat
mengikat
mengikat
mengikat
mengikat
mengikat
mengikat
mengikat
mengikat
mengikat
mengikat
mengikat
mengikat
mengikat
mengikat
mengikat
mengikat
mengikat
mengikat
mengikat
mengikat
mengikat
mengikat
mengikat
mengikat
mengikat
mengisi
mengisi
mengisi
mengisi
mengisi
mengisi
mengisi
mengisi
mengisi
mengisi
mengisi
mengisi
mengisi
mengisi
mengisi
mengisi
mengisi
mengisi
mengisi
mengisi
mengisi
mengisi
mengisi
mengisi
mengisi
mengisi
mengisi
mengisi
mengisi
mengisi
meng

tengah
tengah
tengah
tengah
tengah
tengah
tengah
tengah
tengah
tengah
tinggi
tinggi
tinggi
tinggi
tinggi
tinggi
tinggi
tinggi
tinggi
tinggi
tinggi
tinggi
tinggi
tinggi
tinggi
tinggi
tinggi
tinggi
tinggi
tinggi
tinggi
tinggi
tinggi
tinggi
tinggi
tinggi
tinggi
tinggi
tinggi
tinggi
tinggi
tinggi
tinggi
tinggi
tinggi
tinggi
tinggi
tinggi
tinggi
tinggi
tinggi
tinggi
tinggi
tinggi
tinggi
tinggi
tinggi
tinggi
tinggi
tinggi
tinggi
tinggi
tinggi
tinggi
tinggi
tinggi
tinggi
tinggi
tinggi
tinggi
tinggi
tinggi
tinggi
tinggi
tinggi
tinggi
tinggi
tinggi
tinggi
tinggi
tinggi
tinggi
tinggi
tinggi
tinggi
tinggi
tinggi
tinggi
tinggi
tinggi
tinggi
tinggi
tinggi
tinggi
tinggi
tinggi
tinggi
tinggi
tinggi
tinggi
tinggi
tinggi
tinggi
tinggi
tinggi
tinggi
tinggi
tinggi
tinggi
tinggi
tinggi
tinggi
tinggi
tinggi
tinggi
tinggi
tinggi
tinggi
tinggi
tinggi
tinggi
tinggi
tinggi
tinggi
tinggi
tinggi
tinggi
tinggi
tinggi
tinggi
tinggi
tinggi
tinggi
tinggi
tinggi
tinggi
tinggi
tinggi
tinggi
tinggi
tinggi
tinggi
tinggi

In [22]:
testopd = pd.DataFrame(columns=['kalimat_id', 'kata', 'sense'])
testopd.append({'kalimat_id':'222222',
                'kata':'mantap',
                'sense':'XX01'},ignore_index=True)
testopd.head()

,kalimat_id,kata,sense


In [24]:
all_rows = []
dick = {'kalimat_id':'222222',
                'kata':'mantap',
                'sense':'XX01'}
all_rows.append(dick)

testopd = pd.DataFrame(all_rows,columns=['kalimat_id', 'kata', 'sense'])
testopd

,kalimat_id,kata,sense
0,222222,mantap,XX01
